In [1]:
# This code has to be added to all experiment running notebooks
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with deep_learning_demo


In [4]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip','pandas','numpy'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config
script_config = ScriptRunConfig(source_directory='training_folder',
                                script='training.py',
                                environment=sklearn_env) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='training-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion()

{'runId': 'training-experiment_1681626924_6acf2271',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2023-04-16T06:35:25.066938Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'a0b4d466-1bc9-4e7d-8bfb-285742a38e18'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'sklearn-env',
   'version': 'Autosave_2023-04-15T14:42:37Z_85a3b8f2',
   'assetId': 'azureml://locations/australiaeast/workspaces/7bfba742-3480-47d2-8c40-e65de0d9d12a/environments/sk

In [6]:
# Create a script config
script_config = ScriptRunConfig(source_directory='training_folder',
                                script='training1.py',
                                arguments = ['--reg-rate', 0.1],
                                environment=sklearn_env)

experiment = Experiment(workspace=ws, name='training1-experiment')
run1 = experiment.submit(config=script_config)
run1.wait_for_completion()

{'runId': 'training1-experiment_1681626988_4d21a4c4',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2023-04-16T06:36:29.607262Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'd8c84c25-487f-43ea-85b2-8c6f96753f2b'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'training1.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg-rate', '0.1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'sklearn-env',
   'version': 'Autosave_2023-04-15T14:42:37Z_85a3b8f2',
   'assetId': 'azureml://locations/australiaeast/workspaces/7bfba742-3480-47d2-8c40-e65de0d

In [7]:
# "run" is a reference to a completed experiment run

# List the files generated by the experiment
for file in run.get_file_names():
    print(file)

# Download a named file
run.download_file(name='outputs/model.pkl', output_file_path='model.pkl')

azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/10347_azureml.log
outputs/model.pkl


In [8]:
# "run" is a reference to a completed experiment run

# List the files generated by the experiment
for file in run1.get_file_names():
    print(file)

# Download a named file
run1.download_file(name='outputs/model1.pkl', output_file_path='model1.pkl')

azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/10549_azureml.log
outputs/model1.pkl


In [12]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model1.pkl', # local path
                       description='A classification model',
                       tags={'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Registering model classification_model


In [14]:
run.register_model( model_name='classification_model1',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model',
                    tags={'data-format': 'CSV'},
                    model_framework=Model.Framework.SCIKITLEARN,
                    model_framework_version='0.20.3')

Model(workspace=Workspace.create(name='deep_learning_demo', subscription_id='ba1e8652-16b8-413f-bd32-b36b3804cede', resource_group='deep_learning'), name=classification_model1, id=classification_model1:1, version=1, tags={'data-format': 'CSV'}, properties={})

In [15]:
from azureml.core import Model

for model in Model.list(ws):
    # Get model name and auto-generated version
    print(model.name, 'version:', model.version)

classification_model1 version: 1
classification_model version: 3
classification_model version: 2
classification_model version: 1
amlstudio-predict-penguin-clus version: 1
amlstudio-predict-diabetes version: 1
amlstudio-predict-auto-price-a version: 1
amlstudio-predict-auto-price version: 1
AutoML69f9d88136 version: 3
AutoML69f9d88136 version: 2
AutoML69f9d88136 version: 1
